In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define data directories
train_dir = "/content/drive/MyDrive/DIP/Train"
test_dir = "/content/drive/MyDrive/DIP/Test"

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained VGG16 model
base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)  # Adjusted to dynamically get number of classes

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1280 images belonging to 4 classes.
Found 160 images belonging to 4 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
40/40 [==============================] - 996s 25s/step - loss: 0.9253 - accuracy: 0.5813 - val_loss: 0.6081 - val_accuracy: 0.6938
Epoch 2/10
40/40 [==============================] - 974s 24s/step - loss: 0.5832 - accuracy: 0.6992 - val_loss: 0.5244 - val_accuracy: 0.7375
Epoch 3/10
40/40 [==============================] - 971s 24s/step - loss: 0.5250 - accuracy: 0.7266 - val_loss: 0.4893 - val_accuracy: 0.7000
Epoch 4/10
40/40 [==============================] - 921s 23s/step - loss: 0.5127 - accuracy: 0.7156 - val_loss: 0.5465 - val_accuracy: 0.6687
Epoch 5/10
40/40 [==============================] - 960s 24s/step - loss: 0.4804 - accuracy: 0.7391 - val_loss: 0.4622 - val_accuracy: 0.7250
Epoch 6/10
40